In [ ]:
import sycamore

# Initializng sycamore which also initializes Ray underneath
context = sycamore.init()

from sycamore.transforms.extract_schema import OpenAISchemaExtractor, OpenAIPropertyExtractor
from sycamore.llms import OpenAIModels, OpenAI
import os

openai = OpenAI(OpenAIModels.GPT_3_5_TURBO.value, api_key=os.environ.get("OPENAI_API_KEY"))

from sycamore.transforms.partition import UnstructuredPdfPartitioner

bucket_path = "s3://aryn-public/NTSB/"
sec_docs = context.read.binary(bucket_path, parallelism=1, binary_format="pdf")
sec_docs = sec_docs.limit(5).partition(partitioner=UnstructuredPdfPartitioner())

sec_docs = sec_docs.extract_batch_schema(
    schema_extractor=OpenAISchemaExtractor("AircraftIncident", llm=openai, num_of_elements=35)
).extract_properties(property_extractor=OpenAIPropertyExtractor(llm=openai, num_of_elements=35))

In [ ]:
x = map(lambda doc: doc.properties["entity"], sec_docs.take(5))
list(x)